In [19]:
from astropy.table import Table
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from minisom import MiniSom
import pickle
import os

In [62]:
b = []
for i in range(3, 11):
    dim = i * 10
    for j in range(1, 11):
        sigma = j
        for k in range(1, 11):
            rate = k

            a = {}
            a["dim"] = dim
            a["sigma"] = sigma
            a["rate"] = rate
            b.append(a)

In [20]:
df_L3_info = Table.read('../dataset/L3_COSMOS2020_Richard_RefCat_2023DEC4_info.fits')
df_L3_info = df_L3_info.to_pandas()

In [21]:
L3_phot_data = np.loadtxt('../dataset/L3_COSMOS2020_Richard_RefCat_2023DEC4_averaged_phot102.txt')
phot_data = L3_phot_data[:,3::2]
phot_err_data = L3_phot_data[:,4::2]

In [22]:
data = (phot_data - np.mean(phot_data, axis=0)) / np.std(phot_data, axis=0)

In [77]:
def grid_search(data, file):
    n = 0
    with open(file, "rb") as fh:
        record = pickle.load(fh)
    if "som" not in record[-1]:
        for i in record:
            n += 1
            if "som" not in i:
                som = MiniSom(i["dim"], i["dim"], data.shape[1], sigma = i["sigma"], learning_rate = i["rate"])
                som.random_weights_init(data)
                som.train(data = data, num_iteration = 1, use_epochs = True) 
                i["topo_err"] = som.topographic_error(data)
                i["quan_err"] = som.quantization_error(data)
                i["som"] = som
                i["band_cut"] = len(data[0])
                i["mag_cut"] = len(data)
                i["preproc"] = "default"
                i["topo_err"] = som.topographic_error(data)
                i["quan_err"] = som.quantization_error(data)
                print(f"File: {file}, progress: {round(n / len(record) * 100, 2)}%, param: {[i["dim"], i["sigma"], i["rate"]]}", end = "\r")
                try:
                    with open(file, 'wb') as handle:
                        pickle.dump(record, handle, protocol=pickle.HIGHEST_PROTOCOL)
                except KeyboardInterrupt:
                    print('KeyboardInterrupt caught, data saved.')
    else:
        pass

In [ ]:
for j in os.listdir("D:/SPHEREx_SOM/record/grid_search/"):
    file = "D:/SPHEREx_SOM/record/grid_search/" + j
    grid_search(data = data, file = file)

D:\miniconda\envs\SPHEREx_SOM\Lib\site-packages\numpy\linalg\_linalg.py:2773: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
D:\miniconda\envs\SPHEREx_SOM\Lib\site-packages\numpy\linalg\_linalg.py:2772: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
D:\miniconda\envs\SPHEREx_SOM\Lib\site-packages\minisom.py:346: RuntimeWarning: invalid value encountered in add
  self._weights += einsum('ij, ijk->ijk', g, x-self._weights)
D:\miniconda\envs\SPHEREx_SOM\Lib\site-packages\minisom.py:538: RuntimeWarning: overflow encountered in power
  weights_flat_sq = power(weights_flat, 2).sum(axis=1, keepdims=True)
D:\miniconda\envs\SPHEREx_SOM\Lib\site-packages\numpy\_core\_methods.py:52: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


D:\miniconda\envs\SPHEREx_SOM\Lib\site-packages\minisom.py:540: RuntimeWarning: overflow encountered in multiply
  return sqrt(-2 * cross_term + input_data_sq + weights_flat_sq.T)
D:\miniconda\envs\SPHEREx_SOM\Lib\site-packages\minisom.py:540: RuntimeWarning: invalid value encountered in add
  return sqrt(-2 * cross_term + input_data_sq + weights_flat_sq.T)
